In [ ]:
# Import libs

import psycopg2
import pandas as pd
import numpy as np

In [ ]:
def create_database():
    #connect to default database
    conn = psycopg2.connect("host=127.0.0.1 dbname = postgres user = postgres password = torontoraptors")
    conn.set_session(autocommit=True)
    cur = conn.cursor()

    # Create Sparkify database with UTF8 encoding 
    cur.execute("DROP DATABASE IF EXISTS countries")
    cur.execute("CREATE DATABASE countries")

    #Close connection to default database 
    conn.close()

    #Connect to sparkify database
    conn = psycopg2.connect("host=127.0.0.1 dbname = countries user = postgres password = torontoraptors")
    cur = conn.cursor()

    return cur, conn



In [ ]:
def drop_tables(cur,conn):
    for query in drop_table_queries:
        cur.execute(query)
        conn.commit()

In [ ]:
def create_tables(cur,conn):
    for query in create_table_queries:
        cur.execute(query)
        conn.commit()

In [ ]:
#Create Dataframe
country_data = pd.read_csv("country_profile_variables.csv")

In [ ]:
country_data.head()

In [ ]:
#List all columns
column_headers = list(country_data.columns.values)
print(column_headers)

In [ ]:
#Select certain columns you wish to keep 

country_data_clean = country_data[['country', 'Region', 'Surface area (km2)']]

In [ ]:
country_data_clean.head()

In [ ]:
country_data_clean = country_data_clean.rename(columns={'country':'Country Name'})

In [ ]:
country_data_clean.columns

In [ ]:
#Load in second dataset - Country heights.

country_height = pd.read_csv("country_height_data.csv")



In [ ]:
country_height.columns

In [ ]:
country_height = country_height[['Country Name','Male Height in Ft', 'Female Height in Ft']]

In [ ]:
country_height.head()

In [ ]:
country_death_causes = pd.read_csv("country_death_causes.csv")

In [ ]:
country_death_causes.columns

In [ ]:
country_death_causes = country_death_causes[['Country Name', 'Covid-19 Deaths','Meningitis ', 'Malaria ','HIV/AIDS','Tuberculosis']]

In [ ]:
country_death_causes.head()

In [ ]:
##Create Database "Countries"

cur, conn = create_database()

In [ ]:
#Create first table - country data.
#'Region', 'Surface area (km2)
country_data_table_create = ("""CREATE TABLE IF NOT EXISTS countrydata(
Country_Name VARCHAR PRIMARY KEY,
REGION VARCHAR,
SURFACE_AREA FLOAT
)""")



In [22]:
cur.execute(country_data_table_create)
conn.commit()

In [23]:
#Create second table - country height data
country_height_table_create = ("""CREATE TABLE IF NOT EXISTS countryheight(
Country_Name VARCHAR,
Male_Height_in_Ft INT,
Female_Height_in_Ft INT )""")

cur.execute(country_height_table_create)
conn.commit()

In [24]:
#Create third table - country death causes
country_death_table_create = ("""CREATE TABLE IF NOT EXISTS countrydeaths(
Country_Name VARCHAR,
Covid19_Deaths INT,
Meningitis INT,
Malaria INT, 
HIV_AIDS INT,
Tuberculosis INT
 )""")

cur.execute(country_death_table_create)
conn.commit()

In [25]:
#Insert data in tables

country_data_table_insert = ("""INSERT INTO countrydata(
Country_Name,
REGION,
SURFACE_AREA)
VALUES (%s, %s, %s)
""")

In [26]:
##Having issues with the datatypes in this table
# Need to simplify the dataset. Looks like its the GDP growth rate column giving it problems.

for i, row in country_data_clean.iterrows():
    cur.execute(country_data_table_insert, list(row))

InvalidTextRepresentation: invalid input syntax for type double precision: "~0"
LINE 5: VALUES ('Holy See', 'SouthernEurope', '~0')
                                              ^
